# Prepare Data

### Design Decisions

In [9]:
# Only include instances from SAT competition 202x
# instance_filter_expr = "local like %sat202%"
instance_filter_expr = ""

### Load Data

In [10]:
import numpy as np
import pandas as pd

from gbd_tool.gbd_api import GBD
    
# Meta-data dataframe
with GBD(["gbd-data/meta.db"]) as gbd:
    features = gbd.get_features()
    features.remove("hash")
    meta_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=gbd.get_features())

# Dataframe with all base features
with GBD([
    "gbd-data/meta.db",
    "gbd-data/base.db"
]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    base_features_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
# Dataframe with all features
with GBD([
    "gbd-data/meta.db",
    "gbd-data/base.db",
    "gbd-data/gate.db",
]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    features_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)

# Dataframe with all base and satzilla features
with GBD([
    "gbd-data/meta.db",
    "gbd-data/base.db",
    "gbd-data/gate.db",
    "gbd-data/satzilla.db",
]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    features_satzilla_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
# Dataframe with all runtimes
with GBD(["gbd-data/meta.db", "gbd-data/runtimes.db"]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    runtimes_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
# Handle custom error codes
counts = {}
def convert_value(x: str) -> float:
    global counts
    if x is None or x == "empty" or x == "failed" or x == "memout" or x == "unverified":
        if x in counts:
            counts[x] += 1
        else:
            counts[x] = 1
        return np.nan
    elif x == "timeout":
        return np.inf
    else:
        try:
            return float(x)
        except ValueError as e:
            print(x)
            raise e

# Coerce into correct datatype and set index
for col_name in base_features_df.columns:
    if col_name == "hash":
        base_features_df[col_name] = base_features_df[col_name].astype("string")
    else:
        base_features_df[col_name] = base_features_df[col_name].map(convert_value)   
base_features_df.set_index("hash", inplace=True, drop=True)

for col_name in features_df.columns:
    if col_name == "hash":
        features_df[col_name] = features_df[col_name].astype("string")
    else:
        features_df[col_name] = features_df[col_name].map(convert_value)   
features_df.set_index("hash", inplace=True, drop=True)
   
for col_name in features_satzilla_df.columns:
    if col_name == "hash":
        features_satzilla_df[col_name] = features_satzilla_df[col_name].astype("string")
    else:
        features_satzilla_df[col_name] = features_satzilla_df[col_name].map(convert_value)   
features_satzilla_df.set_index("hash", inplace=True, drop=True)
    
for col_name in runtimes_df.columns:
    if col_name == "hash":
        runtimes_df[col_name] = runtimes_df[col_name].astype("string")
    else:
        runtimes_df[col_name] = runtimes_df[col_name].map(convert_value)  
runtimes_df.set_index("hash", inplace=True, drop=True)

# Only use instances with all feature values present
nan_feature_rows = np.any(np.isnan(features_df.replace([np.inf, -np.inf], np.nan)), axis=1)
base_features_df = base_features_df[~nan_feature_rows]
features_df = features_df[~nan_feature_rows]
features_satzilla_df = features_satzilla_df[~nan_feature_rows]
runtimes_df = runtimes_df[~nan_feature_rows]

# Drop march_nh and glucose_syrup solvers because of too many empty values (not timeout)
runtimes_df.drop(["march_nh", "glucose_syrup"], axis=1, inplace=True)

base_features_df.shape, features_df.shape, features_satzilla_df.shape, runtimes_df.shape

((28225, 56), (28225, 113), (28225, 251), (28225, 16))

In [11]:
# Meta-data dataframe
with GBD(["gbd-data/meta.db"]) as gbd:
    features = gbd.get_features()
    features.remove("hash")
    meta_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=gbd.get_features())

# Dataframe with all base features
with GBD([
    "gbd-data/meta.db",
    "gbd-data/base.db"
]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    base_features_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
# Coerce into correct datatype and set index
for col_name in base_features_df.columns:
    if col_name == "hash":
        base_features_df[col_name] = base_features_df[col_name].astype("string")
    else:
        base_features_df[col_name] = base_features_df[col_name].map(convert_value)   
base_features_df.set_index("hash", inplace=True, drop=True)
meta_df.set_index("hash", inplace=True, drop=True)

# Only use instances with all feature values present
nan_feature_rows = np.any(np.isnan(base_features_df.replace([np.inf, -np.inf], np.nan)), axis=1)
base_features_df = base_features_df[~nan_feature_rows]
meta_df = meta_df.loc[base_features_df.index, :]
meta_df.shape, base_features_df.shape

((29353, 9), (29353, 56))

In [12]:
counts

{None: 2778526, 'memout': 2, 'empty': 212128}

In [13]:
# Drop zero variance features
base_features_df.drop(base_features_df.columns[base_features_df.std() == 0], axis=1, inplace=True)
features_df.drop(features_df.columns[features_df.std() == 0], axis=1, inplace=True)
features_satzilla_df.drop(features_satzilla_df.columns[features_satzilla_df.std() == 0], axis=1, inplace=True)
runtimes_df.drop(runtimes_df.columns[runtimes_df.std() == 0], axis=1, inplace=True)

base_features_df.shape, features_df.shape, features_satzilla_df.shape, runtimes_df.shape

((29353, 46), (28225, 97), (28225, 235), (28225, 16))

### Compute statistics for each feature

In [4]:
pd.set_option("display.max_rows", 300)

In [5]:
features_df.replace([np.inf, -np.inf], np.nan).dropna().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
clauses,2526.0,8.999957e+06,4.770048e+07,5.500000e+01,1.211970e+05,4.789850e+05,3.051494e+06,8.730200e+08
variables,2526.0,4.676456e+05,1.405167e+06,3.200000e+01,1.197600e+04,4.335450e+04,3.197802e+05,2.886922e+07
clause_size_1,2526.0,3.497879e+04,1.238329e+05,0.000000e+00,2.000000e+00,1.480000e+02,1.938000e+03,8.876720e+05
clause_size_2,2526.0,1.296477e+07,4.728255e+07,5.560637e+06,5.567094e+06,5.650376e+06,6.724738e+06,8.785615e+08
clause_size_3,2526.0,1.159643e+06,4.582747e+06,0.000000e+00,8.309250e+03,5.979750e+04,6.084300e+05,9.399184e+07
clause_size_4,2526.0,3.166744e+05,1.596749e+06,0.000000e+00,0.000000e+00,2.745000e+02,3.120800e+04,1.469232e+07
clause_size_5,2526.0,3.503013e+04,3.698999e+05,0.000000e+00,0.000000e+00,0.000000e+00,2.630000e+03,1.572861e+07
clause_size_6,2526.0,6.111778e+06,5.909102e+04,6.101292e+06,6.101292e+06,6.101292e+06,6.104896e+06,7.674150e+06
clause_size_7,2526.0,1.731399e+04,5.839898e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.300000e+02,6.896240e+05
clause_size_8,2526.0,3.132882e+09,5.236574e+08,1.241745e+07,3.276047e+09,3.276047e+09,3.276047e+09,3.279193e+09


In [6]:
stats_per_column = []
for col_name in features_satzilla_df.columns:
    stats_per_column.append(features_satzilla_df[col_name].replace([np.inf, -np.inf], np.nan).dropna().describe())

pd.concat(stats_per_column, axis=1).transpose()

,count,mean,std,min,25%,50%,75%,max
clauses,2526.0,8.999957e+06,4.770048e+07,5.500000e+01,1.211970e+05,4.789850e+05,3.051494e+06,8.730200e+08
variables,2526.0,4.676456e+05,1.405167e+06,3.200000e+01,1.197600e+04,4.335450e+04,3.197802e+05,2.886922e+07
clause_size_1,2526.0,3.497879e+04,1.238329e+05,0.000000e+00,2.000000e+00,1.480000e+02,1.938000e+03,8.876720e+05
clause_size_2,2526.0,1.296477e+07,4.728255e+07,5.560637e+06,5.567094e+06,5.650376e+06,6.724738e+06,8.785615e+08
clause_size_3,2526.0,1.159643e+06,4.582747e+06,0.000000e+00,8.309250e+03,5.979750e+04,6.084300e+05,9.399184e+07
clause_size_4,2526.0,3.166744e+05,1.596749e+06,0.000000e+00,0.000000e+00,2.745000e+02,3.120800e+04,1.469232e+07
clause_size_5,2526.0,3.503013e+04,3.698999e+05,0.000000e+00,0.000000e+00,0.000000e+00,2.630000e+03,1.572861e+07
clause_size_6,2526.0,6.111778e+06,5.909102e+04,6.101292e+06,6.101292e+06,6.101292e+06,6.104896e+06,7.674150e+06
clause_size_7,2526.0,1.731399e+04,5.839898e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.300000e+02,6.896240e+05
clause_size_8,2526.0,3.132882e+09,5.236574e+08,1.241745e+07,3.276047e+09,3.276047e+09,3.276047e+09,3.279193e+09


### Runtime statistics

In [7]:
(
    runtimes_df[col_name].replace([np.inf, -np.inf], 5000).mean(),
    runtimes_df[col_name].replace([np.inf, -np.inf], 5000).sum() / (60 * 60 * 24)
)

(2320.5467300079176, 67.84376203703704)

In [7]:
stats_per_column = []
for col_name in runtimes_df.columns:
    stats_per_column.append(runtimes_df[col_name].replace([np.inf, -np.inf], np.nan).dropna().describe())

pd.concat(stats_per_column, axis=1).transpose()

,count,mean,std,min,25%,50%,75%,max
cadical_elimfalse,1395.0,1122.377412,1351.740051,0.0,73.935,520.77,1853.8400,4987.05
cadical,1457.0,1023.487152,1264.863799,0.0,73.590,432.39,1515.4400,4969.39
cadical_pripro,1519.0,1062.638604,1274.895322,0.0,77.380,473.47,1739.0400,4960.34
cadical_stability,1428.0,1077.792353,1296.114002,0.0,85.365,470.87,1710.8950,4991.42
candy,1075.0,1057.430335,1310.362463,0.0,50.340,417.69,1644.8850,4993.22
glucose_chanseok,1137.0,865.786438,1180.425752,0.0,34.180,286.97,1270.8000,4996.45
glucose,1161.0,923.262274,1241.815299,0.0,43.990,298.47,1412.5500,4995.01
glucose_var_decay099,1100.0,933.588682,1234.385008,0.0,34.715,311.73,1458.3825,4972.54
kissat,1660.0,791.072169,1106.216364,0.0,51.465,268.53,1053.6150,4997.29
lingeling,1229.0,956.540529,1235.901952,0.0,71.080,346.27,1457.5200,4974.17


### PAR-2 Score

In [8]:
pd.DataFrame.from_records(list(sorted(zip(
    runtimes_df.columns,
    np.mean(runtimes_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
kissat,3948.210530
kissat_sweep,3953.243112
lstech_maple,4100.040000
relaxed,4543.060202
slime,4584.955463
cadical_pripro,4625.553460
cadical,4822.336017
cadical_stability,4956.091639
cadical_elimfalse,5097.274937


### Save data

In [16]:
np.all(base_features_df.index == meta_df.index), base_features_df.shape, meta_df.shape

(True, (29358, 46), (29358, 9))

In [17]:
import pickle

with open("./pickled-data/meta_df.pkl", "wb") as file:
    pickle.dump(meta_df.copy(), file)
with open("./pickled-data/base_features_df.pkl", "wb") as file:
    pickle.dump(base_features_df.copy(), file)

In [9]:
import pickle

with open("./pickled-data/meta_df.pkl", "wb") as file:
    pickle.dump(meta_df, file)
with open("./pickled-data/base_features_df.pkl", "wb") as file:
    pickle.dump(base_features_df, file)
with open("./pickled-data/features_df.pkl", "wb") as file:
    pickle.dump(features_df, file)
with open("./pickled-data/features_satzilla_df.pkl", "wb") as file:
    pickle.dump(features_satzilla_df, file)
with open("./pickled-data/runtimes_df.pkl", "wb") as file:
    pickle.dump(runtimes_df, file)

## SAT 2020 and 2021 Main Track Filters

In [9]:
import pickle
import numpy as np
import pandas as pd

from gbd_tool.gbd_api import GBD
    
# Meta-data dataframe
instance_filter_sat_2020 = "track=main_2020"
instance_filter_sat_2021 = "track=main_2021"
with GBD(["gbd-data/meta.db"]) as gbd:
    features = gbd.get_features()
    features.remove("hash")
    sat2020_meta_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_sat_2020, resolve=features),
        columns=gbd.get_features())
    sat2021_meta_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_sat_2021, resolve=features),
        columns=gbd.get_features())
    
main_sat_2020_hashes = sat2020_meta_df["hash"].copy()
main_sat_2021_hashes = sat2021_meta_df["hash"].copy()
main_sat_2020_hashes.shape, main_sat_2021_hashes.shape

((400,), (400,))

In [14]:
with open("./pickled-data/runtimes_df.pkl", "rb") as file:
    runtimes_df = pickle.load(file)
    
sat_202x_submission_hashes = runtimes_df.index.to_series(index=range(runtimes_df.shape[0]))
sat_202x_submission_hashes.shape

(2526,)

In [19]:
import pickle

with open("./pickled-data/instance_filters/main_sat_2020_hashes.pkl", "wb") as file:
    pickle.dump(main_sat_2020_hashes, file)
with open("./pickled-data/instance_filters/main_sat_2021_hashes.pkl", "wb") as file:
    pickle.dump(main_sat_2021_hashes, file)
with open("./pickled-data/instance_filters/sat_202x_submission_hashes.pkl", "wb") as file:
    pickle.dump(sat_202x_submission_hashes, file)

## SAT Competition 2020 / 2021 all solver runtimes

In [54]:
# Dataframe with all runtimes for SAT2020
with GBD(["gbd-data/meta.db", "gbd-data/sc2020.db"]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    runtimes_sc2020_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
counts = {}
for col_name in runtimes_sc2020_df.columns:
    if col_name == "hash":
        runtimes_sc2020_df[col_name] = runtimes_sc2020_df[col_name].astype("string")
    else:
        runtimes_sc2020_df[col_name] = runtimes_sc2020_df[col_name].map(convert_value)  
runtimes_sc2020_df.set_index("hash", inplace=True, drop=True)

runtimes_sc2020_df = runtimes_sc2020_df.loc[main_sat_2020_hashes, :]
runtimes_sc2020_df

,cadical_sc2020,duriansat,exmaple_padc_dl,exmaple_padc_dl_ovau_exp,exmaple_padc_dl_ovau_lin,exmaple_psids_dl,kissat,kissat_sat,kissat_unsat,maple_scavel,...,cadical_trail,cryptominisat_ccnr,cryptominisat_ccnr_lsids,cryptominisat_walksat,exp_l_mld_cbt_dl,exp_v_lgb_mld_cbt_dl,exp_v_l_mld_cbt_dl,exp_v_mld_cbt_dl,glucose3,upglucose_3_padc
hash,,,,,,,,,,,,,,,,,,,,,
0151bedac526ee195bc52e4134cd80e7,4582.5600,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,4850.670,inf,inf,inf,inf,inf,inf
0241f35c5752768d2b0580533d143a14,inf,inf,453.851,455.674,792.756,835.559,inf,1033.9100,inf,3537.570,...,2685.270,608.184,609.657,620.462,1363.640,2240.7100,1371.910,inf,inf,inf
024af9416f8c1dad1b4f974757e38d51,inf,inf,inf,inf,inf,inf,inf,1551.5400,inf,inf,...,inf,1106.640,inf,4761.100,inf,inf,inf,inf,inf,inf
02627689047d06fbb642eef14768d751,90.3281,80.8298,inf,106.447,129.274,1088.920,31.1242,28.2109,inf,623.831,...,313.247,267.281,204.583,566.267,inf,65.1623,inf,620.635,91.3955,153.857
02c6fe8483e4f4474b7ac9731772535d,1026.8200,488.5130,1650.670,1804.430,1752.070,1569.430,332.8930,296.7290,469.5780,2108.490,...,2884.700,522.997,323.419,436.178,3034.470,1967.4500,3644.790,2016.650,inf,4546.570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc538412229f86bc7016b8bc0aca3924,859.2300,566.8060,593.555,562.557,578.860,551.904,625.8010,815.4340,650.0730,614.685,...,1111.830,784.127,840.483,880.968,533.675,567.6820,533.201,612.026,1534.0800,1160.120
fc794887d3aede5fdd422a501f730032,777.9920,711.0130,722.969,712.026,872.827,860.791,684.0720,1337.7100,542.8570,1930.490,...,563.453,616.750,523.208,555.918,2138.700,2019.5100,2167.240,1950.330,1529.4700,1438.540
fcbef8eb7d8cd4f76fc455a25c0d3065,156.0380,3676.9700,913.671,868.995,825.517,674.200,138.8550,254.0870,92.9825,3147.710,...,231.371,914.431,749.614,1179.460,3333.250,2943.6600,3274.050,3099.340,610.8490,807.909


In [55]:
counts

{None: 92544, 'failed': 1211}

In [61]:
runtimes_sc2020_df = runtimes_sc2020_df.loc[:, ~np.any(np.isnan(runtimes_sc2020_df), axis=0)]
runtimes_sc2020_df

,cadical_sc2020,duriansat,exmaple_padc_dl,exmaple_padc_dl_ovau_exp,exmaple_padc_dl_ovau_lin,kissat,kissat_sat,kissat_unsat,maple_scavel,maple_alluip_trail,...,cadical_trail,cryptominisat_ccnr,cryptominisat_ccnr_lsids,cryptominisat_walksat,exp_l_mld_cbt_dl,exp_v_lgb_mld_cbt_dl,exp_v_l_mld_cbt_dl,exp_v_mld_cbt_dl,glucose3,upglucose_3_padc
hash,,,,,,,,,,,,,,,,,,,,,
0151bedac526ee195bc52e4134cd80e7,4582.5600,inf,inf,inf,inf,inf,inf,inf,inf,3110.600,...,inf,inf,inf,4850.670,inf,inf,inf,inf,inf,inf
0241f35c5752768d2b0580533d143a14,inf,inf,453.851,455.674,792.756,inf,1033.9100,inf,3537.570,inf,...,2685.270,608.184,609.657,620.462,1363.640,2240.7100,1371.910,inf,inf,inf
024af9416f8c1dad1b4f974757e38d51,inf,inf,inf,inf,inf,inf,1551.5400,inf,inf,inf,...,inf,1106.640,inf,4761.100,inf,inf,inf,inf,inf,inf
02627689047d06fbb642eef14768d751,90.3281,80.8298,inf,106.447,129.274,31.1242,28.2109,inf,623.831,164.509,...,313.247,267.281,204.583,566.267,inf,65.1623,inf,620.635,91.3955,153.857
02c6fe8483e4f4474b7ac9731772535d,1026.8200,488.5130,1650.670,1804.430,1752.070,332.8930,296.7290,469.5780,2108.490,3290.900,...,2884.700,522.997,323.419,436.178,3034.470,1967.4500,3644.790,2016.650,inf,4546.570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc538412229f86bc7016b8bc0aca3924,859.2300,566.8060,593.555,562.557,578.860,625.8010,815.4340,650.0730,614.685,867.342,...,1111.830,784.127,840.483,880.968,533.675,567.6820,533.201,612.026,1534.0800,1160.120
fc794887d3aede5fdd422a501f730032,777.9920,711.0130,722.969,712.026,872.827,684.0720,1337.7100,542.8570,1930.490,2685.690,...,563.453,616.750,523.208,555.918,2138.700,2019.5100,2167.240,1950.330,1529.4700,1438.540
fcbef8eb7d8cd4f76fc455a25c0d3065,156.0380,3676.9700,913.671,868.995,825.517,138.8550,254.0870,92.9825,3147.710,3443.490,...,231.371,914.431,749.614,1179.460,3333.250,2943.6600,3274.050,3099.340,610.8490,807.909


In [62]:
np.count_nonzero(np.isnan(runtimes_sc2020_df))

0

In [63]:
pd.DataFrame.from_records(list(sorted(zip(
    runtimes_sc2020_df.columns,
    np.mean(runtimes_sc2020_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
kissat_sat,3926.191198
kissat,4070.237330
cryptominisat_ccnr_lsids,4249.319692
cryptominisat_ccnr,4260.106620
cadical_alluip_trail,4428.149656
cadical_alluip,4429.598133
cryptominisat_walksat,4483.774581
cadical_trail,4554.032149
kissat_unsat,4560.038505


In [65]:
# Dataframe with all runtimes for SAT2021
with GBD(["gbd-data/meta.db", "gbd-data/sc2021.db"]) as gbd:
    features = gbd.get_features()
    for meta_feat in gbd.get_features(dbname="meta_db"):
        features.remove(meta_feat)
    features_with_hash = features.copy()
    features_with_hash.insert(0, "hash")
    runtimes_sc2021_df = pd.DataFrame.from_records(
        gbd.query_search(instance_filter_expr, resolve=features),
        columns=features_with_hash)
    
counts = {}
for col_name in runtimes_sc2021_df.columns:
    if col_name == "hash":
        runtimes_sc2021_df[col_name] = runtimes_sc2021_df[col_name].astype("string")
    else:
        runtimes_sc2021_df[col_name] = runtimes_sc2021_df[col_name].map(convert_value)  
runtimes_sc2021_df.set_index("hash", inplace=True, drop=True)

runtimes_sc2021_df = runtimes_sc2021_df.loc[main_sat_2021_hashes, :]
runtimes_sc2021_df

,cadical_hack_gb,CaDiCaL_PriPro,CaDiCaL_PriPro_no_bin,CaDiCaL_rp,CaDiCaL_sc2021,Cadical_SCAVEL01,Cadical_SCAVEL02,CaDiCaL_Stability_HC,cadical_watch_sat,CleanMaple,...,Relaxed_LCFTP,Relaxed_LCFTP_V2,Relaxed_LCFTP_V3,Relaxed_LCMDCBDL_BLB,Relaxed_LCMDCBDL_SCAVEL01,Relaxed_LCMDCBDL_SCAVEL02,SLIME_hess_no_rnd,SLIME_hess_rnd,SLIME_no_hess_no_rnd,SLIME_no_hess_rnd
hash,,,,,,,,,,,,,,,,,,,,,
000a41cdca43be89ed62ea3abf2d0b64,22.4347,9.00577,9.0832,107.783,95.9904,104.243,50.2382,99.731,34.0304,376.825,...,77.78450,38.84350,37.95730,38.17060,38.9252,106.709,86.87150,105.98600,89.22070,40.43220
0041051c73dcdd885d412a38e8b09fba,1488.0000,1016.00000,1022.7700,971.375,201.6360,615.676,413.0990,1066.010,894.6310,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
004b0f451f7d96f6a572e9e76360f51a,261.4790,673.59000,669.4300,974.224,111.5200,409.269,201.8600,inf,117.1340,1465.810,...,53.73250,404.90100,349.05100,1206.83000,113.4300,188.559,271.41900,542.56400,267.06500,73.44250
0107f34364dfff8ab062193b2c71b4de,235.7030,242.10600,248.0670,248.634,93.7477,272.363,358.3750,234.599,256.9330,1536.320,...,362.16500,1180.00000,402.26200,178.38400,201.6650,811.956,293.24000,233.12800,203.81800,179.98100
01135f7a79d555bd0406f00fac22cbe2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fab5698f06861b62c881c77630fbcd4d,1130.6300,1304.93000,1287.2000,518.023,404.5060,1049.650,1949.6900,1408.120,1101.9900,4983.770,...,783.30400,1004.99000,599.72000,1053.28000,906.1090,1039.450,1063.55000,787.87100,565.02300,789.02600
faea4b579b0010b32f001ac4b5b9415a,inf,inf,inf,inf,1026.5200,3539.150,1760.2400,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
fb621b2b4a54ec0f01ac36daaa7f0f5b,896.6090,909.41000,928.9420,833.987,699.4920,757.696,2469.9700,1447.820,884.2750,inf,...,inf,inf,inf,inf,inf,4346.810,inf,inf,inf,inf


In [66]:
counts

{'unverified': 304, 'memout': 1}

In [68]:
runtimes_sc2021_df = runtimes_sc2021_df.loc[:, ~np.any(np.isnan(runtimes_sc2021_df), axis=0)]
runtimes_sc2021_df

,cadical_hack_gb,CaDiCaL_PriPro,CaDiCaL_PriPro_no_bin,CaDiCaL_rp,CaDiCaL_sc2021,Cadical_SCAVEL01,Cadical_SCAVEL02,CaDiCaL_Stability_HC,cadical_watch_sat,cms_expV_gbL,...,Kissat_sc2021_sweep,Maple_simp21,optsat_m21,optsat_R21,ParaFROST,ParaFROST_NoMDM,SLIME_hess_no_rnd,SLIME_hess_rnd,SLIME_no_hess_no_rnd,SLIME_no_hess_rnd
hash,,,,,,,,,,,,,,,,,,,,,
000a41cdca43be89ed62ea3abf2d0b64,22.4347,9.00577,9.0832,107.783,95.9904,104.243,50.2382,99.731,34.0304,8.12578,...,21.1697,16.6443,37.9729,36.5867,121.262000,8.14135,86.87150,105.98600,89.22070,40.43220
0041051c73dcdd885d412a38e8b09fba,1488.0000,1016.00000,1022.7700,971.375,201.6360,615.676,413.0990,1066.010,894.6310,2125.28000,...,286.2280,inf,inf,inf,879.623000,817.84700,inf,inf,inf,inf
004b0f451f7d96f6a572e9e76360f51a,261.4790,673.59000,669.4300,974.224,111.5200,409.269,201.8600,inf,117.1340,351.15800,...,779.4800,83.7475,85.7536,84.9290,60.118400,65.27380,271.41900,542.56400,267.06500,73.44250
0107f34364dfff8ab062193b2c71b4de,235.7030,242.10600,248.0670,248.634,93.7477,272.363,358.3750,234.599,256.9330,112.17600,...,73.0500,169.8680,169.2850,153.2350,172.471000,129.31000,293.24000,233.12800,203.81800,179.98100
01135f7a79d555bd0406f00fac22cbe2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fab5698f06861b62c881c77630fbcd4d,1130.6300,1304.93000,1287.2000,518.023,404.5060,1049.650,1949.6900,1408.120,1101.9900,4261.81000,...,285.6180,502.0150,585.2820,498.3530,1042.520000,658.65200,1063.55000,787.87100,565.02300,789.02600
faea4b579b0010b32f001ac4b5b9415a,inf,inf,inf,inf,1026.5200,3539.150,1760.2400,inf,inf,inf,...,1577.5000,inf,inf,inf,inf,inf,inf,inf,inf,inf
fb621b2b4a54ec0f01ac36daaa7f0f5b,896.6090,909.41000,928.9420,833.987,699.4920,757.696,2469.9700,1447.820,884.2750,inf,...,2017.2200,inf,inf,inf,1448.520000,651.72800,inf,inf,inf,inf


In [69]:
pd.DataFrame.from_records(list(sorted(zip(
    runtimes_sc2021_df.columns,
    np.mean(runtimes_sc2021_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
Kissat_MAB,3194.610178
Kissat_sc2021_sweep,3365.216217
kissat_gb,3366.487862
kissat_crvr_gb,3398.383336
kissat_cf,3508.211038
hKis,3535.051081
cadical_watch_sat,3613.672898
CaDiCaL_PriPro,3627.878053
CaDiCaL_PriPro_no_bin,3628.101647


In [71]:
np.count_nonzero(np.isnan(runtimes_sc2021_df))

0

In [73]:
with open("./pickled-data/runtimes_sc2020_df.pkl", "wb") as file:
    pickle.dump(runtimes_sc2020_df, file)
with open("./pickled-data/runtimes_sc2021_df.pkl", "wb") as file:
    pickle.dump(runtimes_sc2021_df, file)

## Anniversay Data

In [6]:
import numpy as np
import pandas as pd

from gbd_tool.gbd_api import GBD
    
# Meta-data dataframe
# with GBD(["gbd-data/meta.db"]) as gbd:
#     features = gbd.get_features()
#     features.remove("hash")
#     meta_df = pd.DataFrame.from_records(
#         gbd.query_search("", resolve=features),
#         columns=gbd.get_features())
#     meta_df.set_index("hash", inplace=True)
    
meta_df.shape

(29358, 9)

In [7]:
anni_df = pd.read_csv("./anni-seq.csv", index_col="hash")
anni_df = anni_df[anni_df.index.isin(meta_df.index)].copy()
anni_df = anni_df.replace(10000.0, np.inf).copy()
anni_df.shape

(5306, 23)

In [8]:
np.count_nonzero(np.isinf(anni_df))

32171

In [17]:
from sklearn.model_selection import train_test_split

# Assign family label other to small families
anni_meta_df = meta_df.loc[anni_df.index, "family"]
all_families = anni_meta_df.unique()
small_families_mask = anni_meta_df.value_counts(sort=False) < 10
small_families = all_families[small_families_mask]
stratifcation_values = np.where(anni_meta_df.isin(small_families), "other", anni_meta_df)

anni_train_df = train_test_split(
    anni_df, test_size=0.1,
    stratify=stratifcation_values,
    shuffle=True, random_state=42,
)[1]

anni_train_df

,solver0,solver1,solver2,solver3,solver4,solver5,solver6,solver7,solver8,solver9,...,solver13,solver14,solver15,solver16,solver17,solver18,solver19,solver20,solver21,solver22
hash,,,,,,,,,,,,,,,,,,,,,
5c409dd2a3ca6066b7de7d72524b1bec,211.589000,209.058000,382.382000,356.126000,45.299800,101.34000,43.27520,43.579700,143.864000,569.148000,...,286.601000,inf,361.499000,8.550080,51.275300,113.84600,10.10910,101.537000,228.439000,1842.650000
f33b8989cd1bb1390f0891c9f07a9297,4.208620,4.166500,1.720790,8.123540,0.993691,2.21728,9.45244,4.008910,4.253720,6.959190,...,11.692300,4.092760,1.165640,8.988620,5.967610,4.92229,7.92587,0.499766,3.307000,8.103540
3a8409393cc1ebae1444a079d216f221,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
880d9bf224768bf5ff256472682573c8,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
80916cc0ee83c448ca7e1940a0e9e242,583.049000,788.855000,719.921000,476.879000,15.687600,43.35890,343.61000,56.457800,679.248000,112.409000,...,2173.160000,352.134000,107.157000,500.656000,179.494000,565.63400,700.17800,362.371000,425.068000,696.669000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87daaee8aedd931f03bc3577ce787070,295.656000,202.819000,87.802000,139.947000,198.223000,463.85800,130.66000,163.652000,120.484000,252.845000,...,512.281000,133.361000,489.407000,383.161000,143.748000,238.10700,255.88000,664.937000,64.207400,313.376000
a405aa15eb2c944245e36038724303b6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,4664.340000
f34ebd8d4ae8ed75e0062971d7809687,494.106000,657.006000,548.303000,107.355000,331.516000,52.57340,22.89310,332.744000,719.995000,21.548200,...,566.410000,483.374000,111.465000,5.265540,216.766000,110.07000,27.76900,31.244100,33.287700,92.242500


In [19]:
meta_df.loc[anni_df.index, "family"].value_counts()

unknown                         659
hardware-verification           344
cryptography                    308
planning                        252
bitvector                       214
                               ... 
maximum-constraint-partition      2
01-integer-programming            2
circuit-minimization              2
ssp-0                             2
glassy-gen                        1
Name: family, Length: 130, dtype: int64

In [20]:
meta_df.loc[anni_train_df.index, "family"].value_counts()

unknown                     66
hardware-verification       35
cryptography                31
planning                    25
bitvector                   21
                            ..
ordering-principle           1
maxsat-optimum               1
sliding-puzzle               1
automata-synchronization     1
mycielski-graph              1
Name: family, Length: 109, dtype: int64

In [21]:
import pickle

with open("./pickled-data/anni_df.pkl", "wb") as file:
    pickle.dump(anni_df.copy(), file)
with open("./pickled-data/anni_train_df.pkl", "wb") as file:
    pickle.dump(anni_train_df.copy(), file)

In [18]:
np.count_nonzero(np.isinf(anni_train_df))

3291

In [17]:
np.count_nonzero(anni_df.index.isin(base_features_df.index)) / anni_df.shape[0]

1.0

In [14]:
np.count_nonzero(anni_df.index.isin(main_sat_2020_hashes))

354

In [15]:
np.count_nonzero(anni_df.index.isin(main_sat_2021_hashes))

362

In [47]:
r = {}
for v in main_sat_2021_hashes.values:
    c = np.count_nonzero(meta_df["isohash"] == (meta_df.loc[meta_df["hash"] == v, "isohash"].iloc[0]))
    if c in r:
        r[c] += 1
    else:
        r[c] = 1
pd.DataFrame.from_records(list(sorted(r.items())), columns=["Num. Occ. Iso-Hash", "Num. Instance"])

,Num. Occ. Iso-Hash,Num. Instance
0,1,381
1,2,16
2,15,3


In [22]:
pd.DataFrame.from_records(list(sorted(zip(
    anni_df.columns,
    np.mean(anni_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
solver5,2805.617124
solver18,2810.395403
solver12,2832.694092
solver7,2833.073926
solver20,2834.378091
solver11,2842.625442
solver6,2844.245802
solver4,2855.160861
solver19,2866.818358


In [23]:
pd.DataFrame.from_records(list(sorted(zip(
    anni_train_df.columns,
    np.mean(anni_train_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
solver5,2805.801785
solver4,2833.091481
solver6,2845.194253
solver12,2852.985377
solver7,2857.423019
solver18,2865.824259
solver20,2871.853832
solver11,2891.439584
solver17,2894.019080


## Final Anniversary Data

In [16]:
import numpy as np
import pandas as pd

from gbd_tool.gbd_api import GBD
    
# Meta-data dataframe
with GBD(["gbd-data/meta.db"]) as gbd:
    features = gbd.get_features()
    features.remove("hash")
    meta_df = pd.DataFrame.from_records(
        gbd.query_search("", resolve=features),
        columns=gbd.get_features())
    meta_df.set_index("hash", inplace=True)
    
meta_df.shape

(29922, 9)

In [22]:
anni_df = pd.read_csv("./anni-seq-final.csv", index_col="hash", sep=" ")
anni_df.drop(["benchmark", "verified-result", "claimed-result"], axis=1, inplace=True)
anni_df = anni_df[anni_df.index.isin(base_features_df.index)].copy()
anni_df = anni_df.replace(10000.0, np.inf).copy()
anni_df.shape

(5301, 28)

In [23]:
pd.DataFrame.from_records(list(sorted(zip(
    anni_df.columns,
    np.mean(anni_df.replace(np.inf, 10000), axis=0)
), key=lambda x: x[1])), columns=["Algorithm", "PAR-2 score"]).set_index("Algorithm", drop=True)

,PAR-2 score
Algorithm,
Kissat_MAB_ESA,2808.131702
kissat-sc2022-bulky,2812.931160
ekissat-mab-gb-db,2835.247538
Kissat_MAB_UCB,2835.594988
kissat_inc,2836.915263
ekissat-mab-db-v1,2845.189361
Kissat_MAB_MOSS,2846.732671
Kissat_MAB-HyWalk,2857.666290
kissat-sc2022-light,2869.454962


In [24]:
np.count_nonzero(anni_df.index.isin(meta_df.index)), anni_df.shape[0]

(5301, 5301)

In [25]:
np.count_nonzero(anni_df.index.isin(base_features_df.index)), anni_df.shape[0]

(5301, 5301)

In [26]:
with open("./pickled-data/anni_final_df.pkl", "wb") as file:
    pickle.dump(anni_df.copy(), file)

In [4]:
import numpy as np
import pandas as pd
import pickle

anni_full_df = pd.read_csv("./anni-seq-final.csv", index_col="hash", sep=" ")
anni_full_df.drop(["benchmark", "verified-result", "claimed-result"], axis=1, inplace=True)
anni_full_df = anni_full_df.replace(10000.0, np.inf).copy()

with open("./pickled-data/anni_full_df.pkl", "wb") as file:
    pickle.dump(anni_full_df.copy(), file)

anni_full_df.shape

(5355, 28)